In [1]:
import sys  
sys.path.insert(0, '/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/methods_collection/')
import ranking_help_methods as help_methods
import make_question as question_maker
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
TFIDF_vectorizer = TfidfVectorizer()

In [2]:
dataset = pd.read_csv('varieties_corpus/processed_word5perline_corpus.csv')
dataset = dataset.iloc[:,1:]
dataset = dataset.rename(columns={"Text": "Data"})
dataset.PageID = dataset.PageID.apply(lambda x: x.replace("Page_",""))
dataset.Data = dataset.Data.apply(lambda x: help_methods.clean_text(str(x)))
dataset.head()

,PageID,Data
0,0,カスタマイザー タイトル 設定 カスタマイザー サブタイトル
1,0,設定 MobiControl 設定 順序 MobiControl
2,0,設定 順序 MobiControl 設定 順序
3,0,行い 端末 OS 対象 オンプレミス
4,0,MobiControl サーバ 必要 な 設定


In [151]:
dataset.Data.values.tolist()

array(['カスタマイザー タイトル 設定 カスタマイザー サブタイトル',
       '設定 MobiControl 設定 順序 MobiControl', '設定 順序 MobiControl 設定 順序', ...,
       '初期 端末 働きかける 端末 状況', '視る 端末 位置 地図 表示', 'アプリ 活用 状況 監視 0'],
      dtype=object)

In [152]:
def TFIDF_vector_fit(vectorizer, dataset):
    vec = vectorizer.fit(dataset.Data.values.tolist())
    return vec

In [153]:
vec = TFIDF_vector_fit(TFIDF_vectorizer, dataset)

In [3]:
#help_methods.build_voc(dataset)
with open('customized_model/vocabulary.pkl', 'rb') as f:
    word_ids = pickle.load(f)

#help_methods.make_vector(dataset, word_ids)    
with open('customized_model/vectormodel_perpage_v1.pkl', 'rb') as f:
    vector = pickle.load(f)
    
len(vector)   

78244

In [46]:
sample_size = 100
questions_samples = question_maker.question_dataframe_generator_1000(dataset, sample_size)
corpus_per_page = help_methods.corpus_per_page(dataset)

In [48]:
len(corpus_per_page)
corpus_per_page.head()

,Data,PageID
0,カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定 MobiControl ...,0
1,Apple 製品 MobiControl v ManualApple 製品 MobiCont...,1
2,Android 端末 MobiControl v ManualAndroid 設定 モード ...,2
3,Windows PCMobiControl v ManualWindows PC 根本 セキ...,3
4,Windows EmbeddedMobiControl v ManualWindows Em...,4


In [5]:
for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
#     input_vector = list(set(str(query).split()))
#     query_vector = help_methods.custom_vectorizer(input_vector, word_ids)
    break

managed アカウント アカウント 取得 使用しますか 252


In [19]:
    question_parts = question_maker.making_query_collection(query)
    collector = help_methods.sequence_searcher(corpus_per_page, question_parts)

Debug for Collector:  ['391', '387', '51', '0', '252', '92', '229', '236', '368', '386', '2', '431', '147', '369', '438', '385', '429', '432', '427', '14']


In [32]:
question_parts

['managed アカウント',
 'managed アカウント アカウント',
 'アカウント アカウント',
 'アカウント アカウント 取得',
 'アカウント 取得',
 'アカウント 取得 使用しますか',
 '取得 使用しますか']

In [12]:
merged = pd.read_csv('../../Helpers/Title_link_merged_corpus.csv')
merged = merged.iloc[:,2:]
merged = merged.rename(columns={"Article": "Data"})
merged.head()


,Data,Summary,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,MobiControl v14 Manual – MobiControl v14 Manual,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,Apple製品 – MobiControl v14 Manual ios overall,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,Android端末 – MobiControl v14 Manual plus overall,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,Windows PC – MobiControl v14 Manual wpc overall,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,Windows Embedded – MobiControl v14 Manual wm o...,4


In [66]:
q = query.split()
q = [items for items in q if items not in question_maker.get_questions_delimiter_ja()] 
q = ' '.join(q)

In [144]:
q

'managed アカウント アカウント 取得'

In [113]:
from difflib import SequenceMatcher
rat = []
smr = []
result = []
for pages_id in collector:
    page_data = corpus_per_page[(corpus_per_page['PageID']==str(pages_id))].Data.values
    result.append([help_methods.sequence_matcher(str(q), str(page_data)), pages_id, rat])    
    

In [156]:
res = []
freq_words = ['使', 'し', 'ま', 'す', 'か', 'しま', 'カ']
for items, ids, rat in result:
    contents = [terms for terms in items if terms not in freq_words]
    contents = [temp.strip(' ') for temp in contents]
    if len(contents) > 1:
        #res.append([ids, ' '.join(contents)])
        res.append(ids)

In [ ]:
#sorted(res, key=lambda l:l[1] , reverse=True)

In [175]:
match_collection = []
for ids in res:
    page_data = dataset[(dataset['PageID']==str(ids))].Data.values
    for lines in page_data:
        for parts in question_parts:
            if lines.find(parts) !=-1:
                match_collection.append([ids, lines])

In [ ]:
sorted_list = help_methods.get_unique_2Dlist(match_collection)

In [208]:
X = vec.transform([str(query)])
rank = []
ids_list = []
for ids, items in sorted_list:
    for sentences in items:
        #print(sentences)
        Y = vec.transform([sentences])
        ids_list.append(ids)
        rank.append(cosine_similarity(X, Y))

In [209]:
    flat = [x for sublist in rank for x in sublist]
    result = sorted(zip(ids_list, flat), key=lambda l:l[1], reverse=True)[:3]
    result

[('252', array([0.93032916])),
 ('252', array([0.75009422])),
 ('229', array([0.73883448]))]

In [204]:
[str(query)]

['managed アカウント アカウント 取得 使用しますか']

In [73]:
corpus_fit = list(dataset.Data.values)

In [93]:
def ranking_result_collection(query, text_collections, ids_list, vec):
    X = vec.transform([str(query)])

    rank=[]
    for contents in text_collections:
        Y = vec.transform([contents])
        rank.append(cosine_similarity(X, Y))
    flat = [x for sublist in rank for x in sublist]
    result = sorted(zip(ids_list, flat), key=lambda l:l[1], reverse=True)[:10]
    return result